In [1]:
from caveclient import CAVEclient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from standard_transform import minnie_ds
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

datastack_name = 'minnie65_public'
client = CAVEclient(datastack_name)

# Show the description of the datastack
client.info.get_datastack_info()['description']

# see the available materialization versions
client.materialize.get_versions()

versions = sorted(client.materialize.get_versions(), key = lambda v: client.materialize.get_timestamp(v))

for version in versions:
    print(f"Version {version}: {client.materialize.get_timestamp(version)}")

# CAVE always use the same version for consistency
client.version = 1621

Version 117: 2021-06-11 08:10:00.215114+00:00
Version 343: 2022-02-24 08:10:00.184668+00:00
Version 661: 2023-04-06 20:17:09.199182+00:00
Version 795: 2023-08-23 08:10:01.404268+00:00
Version 943: 2024-01-22 08:10:01.497934+00:00
Version 1078: 2024-06-05 10:10:01.203215+00:00
Version 1181: 2024-09-16 10:10:01.121167+00:00
Version 1300: 2025-01-13 10:10:01.286229+00:00
Version 1412: 2025-04-29 10:10:01.200893+00:00
Version 1507: 2025-07-31 08:10:01.117494+00:00
Version 1621: 2025-11-25 08:10:01.094430+00:00


In [2]:
client.materialize.get_tables()

['baylor_gnn_cell_type_fine_model_v2',
 'vortex_manual_myelination_v0',
 'synapse_target_predictions_ssa',
 'aibs_metamodel_celltypes_v661',
 'nucleus_alternative_points',
 'allen_column_mtypes_v2',
 'bodor_pt_cells',
 'aibs_metamodel_mtypes_v661_v2',
 'vortex_microglia_proofreading_status',
 'proofreading_status_and_strategy',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'apl_functional_coreg_vess_fwd',
 'vortex_axon_backtrace_column',
 'vortex_compartment_targets',
 'baylor_log_reg_cell_type_coarse_v1',
 'vortex_synapse_reattachment',
 'gamlin_2023_mcs_met_types',
 'gamlin_2023_mcs',
 'l5et_column',
 'pt_synapse_targets',
 'coregistration_manual_v4',
 'cg_cell_type_calls',
 'synapses_pni_2',
 'nucleus_detection_v0',
 'vortex_manual_nodes_of_ranvier',
 'bodor_pt_target_proofread',
 'nucleus_functional_area_assignment',
 'coregistration_auto_phase3_fwd_apl_vess_combined_v2',
 'vortex_thalamic_proofreading_status',
 'multi_input_spine

In [3]:
client.materialize.get_table_metadata('aibs_metamodel_celltypes_v661')

{'id': 80383,
 'aligned_volume': 'minnie65_phase3',
 'table_name': 'aibs_metamodel_celltypes_v661',
 'valid': True,
 'schema': 'cell_type_reference',
 'created': '2023-08-17T05:43:27.032456',
 'schema_type': 'cell_type_reference',
 'user_id': '1077',
 'description': 'This table contains anatomical cell-type predictions for cells throughout the entire dataset at materialization version 661. The predictions come from a soma and nucleus feature trained metamodel (Elabbady 2022). This is a reference table where id refers to the unique nucleus id in the "nucleus_detection_v0" table. Classification_system refers to the coarse class predictions and cell_type denotes nonneuronal class and neuronal subclass predictions. Errors and soma-soma mergers have been filtered out. For questions please contact Leila Elabbady or Forrest Collman. [Note: This table \'aibs_metamodel_celltypes_v661\' will update the \'target_id\' foreign_key when updates are made to the \'nucleus_detection_v0\' table] ',
 'no

In [4]:
# Proofread Connectivity of one cell type in one box


# 23P


from functions import bounding_box

bounding_box_proofread = bounding_box(600, 140, 800, 700, 240, 900)

cell_type_single = '23P'

cell_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_single, pt_position_bbox = bounding_box_proofread).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_list = cell_df['pt_root_id'].tolist()
proofread_df = client.materialize.tables.proofreading_status_and_strategy(pt_root_id = cell_list).query(split_positions = True)
proofread_list = proofread_df['valid_id'].tolist()
number_cells = len(proofread_list)
print('number proofread cells within box:', number_cells)

output_syn_df = client.materialize.synapse_query(pre_ids = proofread_list, post_ids = proofread_list)
output_df = output_syn_df.groupby(['pre_pt_root_id', 'post_pt_root_id']).count()[['id']].rename(columns = {'id': 'syn_count'}).sort_values(by = 'syn_count', ascending = False,).reset_index()    
print(output_df)

unique_connections_df = output_df[['pre_pt_root_id', 'post_pt_root_id']].drop_duplicates()
number_connections_found = len(unique_connections_df)

print('number_connections_found (unique pairs):', number_connections_found)

number_cells_total = number_cells
print('number_cells_total:', number_cells_total)
total_possible_connections = (number_cells_total) * (number_cells_total - 1)
print('total_possible_connections:', total_possible_connections)

connectivity = (number_connections_found / total_possible_connections)
connectivity_percent = connectivity * 100
print(f'connectivity: {connectivity_percent:.4}%')

reciprocity_rate_chance = (connectivity * connectivity) * 100
print(f'reciprocity by chance: {reciprocity_rate_chance:.4}%')

voxels: x=[161123], y=[120584], z=[20000]
voxels: x=[188206], y=[143310], z=[22500]
number proofread cells within box: 142
          pre_pt_root_id     post_pt_root_id  syn_count
0     864691136674327431  864691136108332249          4
1     864691135349839831  864691135801170018          4
2     864691135867009413  864691136330407914          4
3     864691135361314119  864691135779587517          4
4     864691135562842337  864691135885921648          4
...                  ...                 ...        ...
1196  864691134966014495  864691135584504429          1
1197  864691134966014495  864691135756145106          1
1198  864691134966014495  864691136311114330          1
1199  864691134966014495  864691136330407914          1
1200  864691134966495519  864691135277186789          1

[1201 rows x 3 columns]
number_connections_found (unique pairs): 1201
number_cells_total: 142
total_possible_connections: 20022
connectivity: 5.998%
reciprocity by chance: 0.3598%


In [6]:
# Proofread Connectivity within one box and two different cell type


#PN23-> BC


from functions import bounding_box
    
bounding_box_twotypes = bounding_box(600, 140, 800, 700, 240, 900)

cell_type_one = '23P'
cell_type_two = 'BC'

cell_type_one_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_one, pt_position_bbox = bounding_box_twotypes).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_type_one_list = cell_type_one_df['pt_root_id'].tolist()
proofread_one_df = client.materialize.tables.proofreading_status_and_strategy(pt_root_id = cell_type_one_list).query(split_positions = True)
proofread_one_list = proofread_one_df['valid_id'].tolist()
number_cell_type_one = len(proofread_one_list)
print(f'number {cell_type_one}: {number_cell_type_one}')

cell_type_two_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_two, pt_position_bbox = bounding_box_twotypes).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_type_two_list = cell_type_two_df['pt_root_id'].tolist()

proofread_two_df = client.materialize.tables.proofreading_status_and_strategy(pt_root_id = cell_type_two_list).query(split_positions = True)
proofread_two_list = proofread_two_df['valid_id'].tolist()

number_cell_type_two = len(proofread_two_list)
print(f'number {cell_type_two}: {number_cell_type_two}')

combined_df = pd.concat([proofread_one_df, proofread_two_df], ignore_index=True)

output_syn_df = client.materialize.synapse_query(pre_ids = proofread_one_list, post_ids = proofread_two_list)
output_df = output_syn_df.groupby(['pre_pt_root_id', 'post_pt_root_id']).count()[['id']].rename(columns = {'id': 'syn_count'}).sort_values(by = 'syn_count', ascending = False,).reset_index()    
print(output_df)

unique_connections_df = output_df[['pre_pt_root_id', 'post_pt_root_id']].drop_duplicates()
number_connections_found = len(unique_connections_df)

print('number_connections_found (unique pairs):', number_connections_found)

number_cells_total = number_cell_type_one + number_cell_type_two
print('number_cells_total:', number_cells_total)
total_possible_connections = (number_cell_type_one) * (number_cell_type_two)
print('total_possible_connections:', total_possible_connections)

connectivity = (number_connections_found / total_possible_connections)
connectivity_percent = connectivity * 100
print(f'connectivity: {connectivity_percent:.4}%')

reciprocity_rate_chance = (connectivity * connectivity) * 100
print(f'reciprocity by chance: {reciprocity_rate_chance:.4}%')

voxels: x=[161123], y=[120584], z=[20000]
voxels: x=[188206], y=[143310], z=[22500]
number 23P: 142
number BC: 7
         pre_pt_root_id     post_pt_root_id  syn_count
0    864691135304641703  864691136010404012         16
1    864691136488821266  864691135939380228          9
2    864691135988769027  864691136138803837          7
3    864691135388024449  864691136010404012          7
4    864691136138583677  864691136010404012          7
..                  ...                 ...        ...
313  864691136968429774  864691136138803837          1
314  864691135067863108  864691135939380228          1
315  864691134966495519  864691136619344859          1
316  864691134966495519  864691136138803837          1
317  864691136991328917  864691136619344859          1

[318 rows x 3 columns]
number_connections_found (unique pairs): 318
number_cells_total: 149
total_possible_connections: 994
connectivity: 31.99%
reciprocity by chance: 10.23%


In [5]:
# NOT Proofread Connectivity of one cell type in one box


# BC

from functions import bounding_box

bounding_box_proofread = bounding_box(600, 140, 750, 700, 240, 900)
cell_type_single = 'BC'

cell_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_single, pt_position_bbox = bounding_box_proofread).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_list = cell_df['pt_root_id'].tolist()

number_cells = len(cell_list)
print('number cells within box:', number_cells)

output_syn_df = client.materialize.synapse_query(pre_ids = cell_list, post_ids = cell_list)
output_df = output_syn_df.groupby(['pre_pt_root_id', 'post_pt_root_id']).count()[['id']].rename(columns = {'id': 'syn_count'}).sort_values(by = 'syn_count', ascending = False,).reset_index()
print(output_df)

unique_connections_df = output_df[['pre_pt_root_id', 'post_pt_root_id']].drop_duplicates()
number_connections_found = len(unique_connections_df)

print('number_connections_found (unique pairs):', number_connections_found)

number_cells_total = number_cells
print('number_cells_total:', number_cells_total)
total_possible_connections = (number_cells_total) * (number_cells_total - 1)
print('total_possible_connections:', total_possible_connections)

connectivity = (number_connections_found / total_possible_connections)
connectivity_percent = connectivity * 100
print(f'connectivity: {connectivity_percent:.4}%')

reciprocity_rate_chance = (connectivity * connectivity) * 100
print(f'reciprocity by chance: {reciprocity_rate_chance:.4}%')

voxels: x=[161123], y=[120584], z=[18750]
voxels: x=[188206], y=[143310], z=[22500]
number cells within box: 11
        pre_pt_root_id     post_pt_root_id  syn_count
0   864691135579134981  864691136619344859         16
1   864691135448017554  864691135106372685         15
2   864691136484886572  864691135740572564         14
3   864691136484886572  864691135106372685         14
4   864691135939380228  864691136010404012         12
..                 ...                 ...        ...
73  864691136010404012  864691136619344859          1
74  864691136138803837  864691136484886572          1
75  864691136619344859  864691135106372685          1
76  864691136619344859  864691135448017554          1
77  864691136619344859  864691135295126028          1

[78 rows x 3 columns]
number_connections_found (unique pairs): 78
number_cells_total: 11
total_possible_connections: 110
connectivity: 70.91%
reciprocity by chance: 50.28%


In [7]:
# NOT Proofread Connectivity within one box and two different cell type


#BC-> PN23


from functions import bounding_box
    
bounding_box_twotypes = bounding_box(600, 140, 800, 700, 240, 900)

cell_type_one = 'BC'
cell_type_two = '23P'

cell_type_one_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_one, pt_position_bbox = bounding_box_twotypes).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_type_one_list = cell_type_one_df['pt_root_id'].tolist()

number_cell_type_one = len(cell_type_one_list)
print(f'number {cell_type_one}: {number_cell_type_one}')

cell_type_two_df = client.materialize.tables.aibs_metamodel_celltypes_v661(cell_type = cell_type_two, pt_position_bbox = bounding_box_twotypes).query(split_positions = True) # split positions true to prepare for further steps (coordinates)
cell_type_two_list = cell_type_two_df['pt_root_id'].tolist()

number_cell_type_two = len(cell_type_two_list)
print(f'number {cell_type_two}: {number_cell_type_two}')

combined_df = pd.concat([proofread_one_df, proofread_two_df], ignore_index=True)

output_syn_df = client.materialize.synapse_query(pre_ids = cell_type_one_list, post_ids = cell_type_two_list)
output_df = output_syn_df.groupby(['pre_pt_root_id', 'post_pt_root_id']).count()[['id']].rename(columns = {'id': 'syn_count'}).sort_values(by = 'syn_count', ascending = False,).reset_index()    
print(output_df)


unique_connections_df = output_df[['pre_pt_root_id', 'post_pt_root_id']].drop_duplicates()
number_connections_found = len(unique_connections_df)

print('number_connections_found (unique pairs):', number_connections_found)

number_cells_total = number_cell_type_one + number_cell_type_two
print('number_cells_total:', number_cells_total)
total_possible_connections = (number_cell_type_one) * (number_cell_type_two)
print('total_possible_connections:', total_possible_connections)

connectivity = (number_connections_found / total_possible_connections)
connectivity_percent = connectivity * 100
print(f'connectivity: {connectivity_percent:.4}%')

reciprocity_rate_chance = (connectivity * connectivity) * 100
print(f'reciprocity by chance: {reciprocity_rate_chance:.4}%')

voxels: x=[161123], y=[120584], z=[20000]
voxels: x=[188206], y=[143310], z=[22500]
number BC: 7
number 23P: 198
         pre_pt_root_id     post_pt_root_id  syn_count
0    864691135939380228  864691135489677882         24
1    864691135939380228  864691135698708501         23
2    864691135939380228  864691135938604804         20
3    864691135939380228  864691135350981847         18
4    864691135448017554  864691135361939527         17
..                  ...                 ...        ...
656  864691135448017554  864691135164707629          1
657  864691135448017554  864691135156584292          1
658  864691136619344859  864691136579880980          1
659  864691135448017554  864691134966014495          1
660  864691136619344859  864691136904594610          1

[661 rows x 3 columns]
number_connections_found (unique pairs): 661
number_cells_total: 205
total_possible_connections: 1386
connectivity: 47.69%
reciprocity by chance: 22.74%
